# Конечные автоматы

Определение и интуитивные представления см. в [википедии](https://ru.wikipedia.org/wiki/%D0%94%D0%B5%D1%82%D0%B5%D1%80%D0%BC%D0%B8%D0%BD%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%BD%D1%8B%D0%B9_%D0%BA%D0%BE%D0%BD%D0%B5%D1%87%D0%BD%D1%8B%D0%B9_%D0%B0%D0%B2%D1%82%D0%BE%D0%BC%D0%B0%D1%82) и в книге:
* Д. Хопкрофт, Р. Мотвани, and Д. Ульман, Введение в теорию автоматов, языков и вычислений.

In [8]:
"""
Абстрактный тип, объединяющий все конечные автоматы.
S -- state, L -- letter.
"""
abstract type FiniteAutomaton{S, L} end


"""
Детерминированный конечный автомат
Deterministic Finite Automaton
"""
struct DFA{S, L} <: FiniteAutomaton{S, L}
    "Метки состояний автомата, состояние задаётся индексом массива"
    states::Vector{S}
    state2index::Dict{S, Int}
    "Алфавит языка"
    alphabet::Set{L}
    "Функция перехода. Текущему состоянию и текущему входному символу сопоставляется следующее состояние"
    transition::Dict{Int, Dict{L, Int}}
    "Начальное состояние"
    start::Int
    "Множество допускающих состояний"
    accept::Set{Int}
end


"""
Получить множество состояний из переходной функции, начального и допускающих состояний.
"""
function collect_states(
        transition::Dict{S, Dict{L, W}},
        start::S,
        accept::Set{S})::Tuple{Vector{S}, Dict{S, Int}} where {S, L, W}

    # Собираем все состояния из переходной функции. У DFA и NFA эти функции разные, поэтому абстрагируем.
    stateset = collect_transition_states(transition, start, accept)

    # Добавляем начальное и допускающие состояния (если их ещё нет)
    push!(stateset, start)
    union!(stateset, accept)
    
    # Если можно сравнивать имена состояний, то сортируем их.
    # Иначе укладываем в сложившемся порядке в массив.
    states = collect(stateset)::Vector{S}
    hasmethod(isless, Tuple{S, S}) && sort!(states)

    # На основе массива делаем словарь state2index
    state2index = Dict((n, i) for (i, n) in enumerate(states))
    
    return (states, state2index)
end


"""
Собрать состояния из переходной функции
"""
function collect_transition_states(
        transition::Dict{S, Dict{L, S}},
        start::S,
        accept::Set{S})::Set{S} where {S, L}
    
    states = Set{S}(keys(transition))::Set{S}
    for state in keys(transition)
        union!(states, values(transition[state]))
    end
    
    return states
end


"""
Собрать алфавит из переходной функции
"""
function collect_alphabet(transition::Dict{S, Dict{L, W}})::Set{L} where {S, L, W}
    # Собираем из переходной функции алфавит
    alphabet = Set{L}()
    for trans in values(transition)
        union!(alphabet, keys(trans))
    end
    
    return alphabet
end


"""
Поменять в функции перехода имена состояний на их индексы
"""
function map_transition(
        transition::Dict{S, Dict{L, S}},
        state2index::Dict{S, Int})::Dict{Int, Dict{L, Int}} where {S, L}
    
    transition_pairs =
      [(state2index[f], Dict{L, Int}((l, state2index[t]) for (l, t) in l2s))
        for (f, l2s) in transition] ::Vector{Tuple{Int, Dict{L, Int}}}

    return Dict{Int, Dict{L, Int}}(transition_pairs)
end


"""
Дополнительный конструктор, который сам выясняет алфавит и множество состояний
"""
function DFA(transition::Dict{S, Dict{L, S}}, start::S, accept::Set{S})::DFA{S, L} where {S, L}
    (states, state2index) = collect_states(transition, start, accept)
    alphabet = collect_alphabet(transition)
    transition_ = map_transition(transition, state2index)

    start_ = state2index[start]
    accept_ = Set{Int}(state2index[s] for s in accept)

    return DFA{S, L}(states, state2index, alphabet, transition_, start_, accept_)
end


"""
Содержит ли язык данное слово?
"""
function Base.in(word, dfa::DFA{S, L})::Bool where {S, L}
    state = dfa.start
    for symbol in word
        state = dfa.transition[state][symbol]
    end
    
    return state in dfa.accept
end

Base.in

In [9]:
# Пример конечного автомата, допускающего все слова, содержащие "01"

dfa = DFA(Dict(:a => Dict('0' => :b, '1' => :a), :b => Dict('0' => :b, '1' => :c), :c => Dict('0' => :c, '1' => :c)), :a, Set([:c]))

"0000010" ∈ dfa, "10000" ∈ dfa

(true, false)

In [22]:
# Пример конечного автомата, допускающего все слова с нечётным числом нулей

dfa = DFA(Dict(1 => Dict('0' => 2, '1' => 1), 2 => Dict('0' => 1, '1' => 2)), 1, Set([2]))

"111111" ∈ dfa, "0101101011001110111" ∈ dfa

(false, true)

In [47]:
"""
Недетерминированный конечный автомат. Поля аналогичны DFA, кроме функции перехода.
"""
struct NFA{S, L} <: FiniteAutomaton{S, L}
    "Метки состояний автомата, состояние задаётся индексом массива"
    states::Vector{S}
    state2index::Dict{S, Int}
    "Алфавит языка"
    alphabet::Set{L}
    "Функция перехода. Текущему состоянию и текущему входному символу сопоставляется множество следующих состояний"
    transition::Dict{Int, Dict{L, Set{Int}}}
    "Начальное состояние"
    start::Int
    "Множество допускающих состояний"
    accept::Set{Int}
end


"""
Собрать состояния из переходной функции. Метод для функции перехода NFA
"""
function collect_transition_states(
        transition::Dict{S, Dict{L, Set{S}}},
        start::S,
        accept::Set{S})::Set{S} where {S, L}

    states = Set{S}(keys(transition))::Set{S}
    for state in keys(transition)
        for targets in values(transition[state])
            union!(states, targets)
        end
    end
    
    return states
end


"""
Поменять в функции перехода имена состояний на их индексы
"""
function map_transition(
        transition::Dict{S, Dict{L, Set{S}}},
        state2index::Dict{S, Int})::Dict{Int, Dict{L, Set{Int}}} where {S, L}
    
    transition_pairs =
      [(state2index[f], Dict{L, Set{Int}}((l, Set{Int}(state2index[t] for t in ss)) for (l, ss) in l2ss))
        for (f, l2ss) in transition] ::Vector{Tuple{Int, Dict{L, Set{Int}}}}

    return Dict{Int, Dict{L, Set{Int}}}(transition_pairs)
end


"""
Дополнительный конструктор, который сам выясняет алфавит и множество состояний
"""
function NFA(transition::Dict{S, Dict{L, Set{S}}}, start::S, accept::Set{S})::NFA{S, L} where {S, L}
    (states, state2index) = collect_states(transition, start, accept)
    alphabet = collect_alphabet(transition)
    transition_ = map_transition(transition, state2index)

    start_ = state2index[start]
    accept_ = Set{Int}(state2index[s] for s in accept)

    return NFA{S, L}(states, state2index, alphabet, transition_, start_, accept_)
end


"""
Содержит ли язык данное слово?
"""
function Base.in(word, nfa::NFA{S, L})::Bool where {S, L}
    states = Set{S}()
    push!(states, nfa.start)
    for symbol in word
        states = union([get(nfa.transition[state], symbol, Set{S}()) for state in states]...)::Set{S}
    end
    
    return ~isempty(intersect(states, nfa.accept))
end

Base.in

In [50]:
# Пример конечного автомата, допускающего все слова, окончавающиеся на 1

nfa = NFA(Dict{Int, Dict{Char, Set{Int}}}(1 => Dict('0' => Set(1), '1' => Set([1, 2])), 2 => Dict()), 1, Set([2]))
"000010" ∈ nfa, "111001001" ∈ nfa

(false, true)

In [52]:
"""
Преобразование DFA -> NFA
"""
function NFA(dfa::DFA{S, L})::NFA{S, L} where {S, L}
    trans = Dict(from => Dict(sym => Set(to) for (sym, to) in pairs(perstate))
                 for (from, perstate) in pairs(dfa.transition))

    return NFA{S, L}(dfa.states, dfa.state2index, dfa.alphabet, trans, dfa.start, dfa.accept)
end

NFA

In [ ]:
"""
Недетерминированный конечный автомат с эпсилон-переходами.
Поля аналогичны NFA, кроме эпсилон-переходов, заданных в поле epsilon.
"""
struct EpsilonNFA{S, L} <: FiniteAutomaton{S, L}
    "Метки состояний автомата, состояние задаётся индексом массива"
    states::Vector{S}
    state2index::Dict{S, Int}
    "Алфавит языка"
    alphabet::Set{L}
    "Функция перехода. Текущему состоянию и текущему входному символу сопоставляется множество следующих состояний"
    transition::Dict{Int, Dict{L, Set{Int}}}
    epsilon::Dict{Int, Set{Int}}
    "Начальное состояние"
    start::Int
    "Множество допускающих состояний"
    accept::Set{Int}
end


"""
Дополнительный конструктор, который сам выясняет алфавит и множество состояний
"""
function EpsilonNFA(
        transition::Dict{S, Dict{L, Set{S}}},
        epsilon::Dict{S, Set{S}},
        start::S,
        accept::Set{S})::NFA{S, L} where {S, L}
    (states, state2index) = collect_states(transition, start, accept)
    alphabet = collect_alphabet(transition)
    transition_ = map_transition(transition, state2index)
    epsilon_ = Dict{S, Set{S}}((state2index[s], Set(state2index[t] for t in ss)) for s in (s, ss))

    start_ = state2index[start]
    accept_ = Set{Int}(state2index[s] for s in accept)

    return EpsilonNFA{S, L}(states, state2index, alphabet, transition_, epsilon_, start_, accept_)
end

In [ ]:
"""
Преобразование NFA -> EpsilonNFA
"""
function EpsilonNFA(nfa::NFA{S, L})::EpsilonNFA{S, L} where {S, L}
    # ЗДЕСЬ КОД
end

In [ ]:
"""
Преобразование EpsilonNFA -> DFA
"""
function DFA(nfa::EpsilonNFA{S, L})::DFA{S, L} where {S, L}
    # ЗДЕСЬ КОД
end

In [5]:
"""
Регулярные выражения
"""
abstract type RegularExpression{L} end

const RE{L} = RegularExpression{L}

"""
Пустое регулярное выражение
"""
struct EmptyRE{L} <: RE{L}
end


"""
Регулярное выражение из одной буквы алфавита
"""
struct LetterRE{L} <: RE{L}
    value::L
end


"""
Регулярное выражение, полученное конкатенацией
"""
struct ConcatRE{L} <: RE{L}
    values::Vector{RE{L}}
end


"""
Регулярные выражение, полученное объединением
"""
struct UnionRE{L} <: RE{L}
    values::Vector{RE{L}}
end


"""
Регулярное выражение -- звезда Клини
"""
struct KleeneRE{L} <: RE{L}
    value::RE{L}
end

;

In [7]:
# Пример регулярного выражения, описывающего язык из двоичных слов, оканчивающихся на 1
zeroOrOne = UnionRE{Char}(RE{Char}[LetterRE{Char}('0'), LetterRE{Char}('1')])
re = ConcatRE{Char}(RE{Char}[KleeneRE{Char}(zeroOrOne), LetterRE{Char}('1')])

# Текстом такое выражение обычно записывается как (0|1)*1

ConcatRE{Char}(RegularExpression{Char}[KleeneRE{Char}(UnionRE{Char}(RegularExpression{Char}[LetterRE{Char}('0'), LetterRE{Char}('1')])), LetterRE{Char}('1')])

In [ ]:
"""
Преобразование RE -> EpsilonNFA
"""
function EpsilonNFA(re::RE{L})::EpsilonNFA{Int, L} where L
    # ЗДЕСЬ КОД
end

In [ ]:
"""
Преобразование DFA -> RE
"""
function RE(dfa::DFA{S, L})::RE{L} where L
    # ЗДЕСЬ КОД
end